In [1]:
import config
import utils

import json
import pickle
import os
import random
import numpy as np

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.patches as patches

import matplotlib
matplotlib.use('agg')

In [2]:
video_idx = [v_idx for v_idx in os.listdir(config.mocap_dir) if v_idx[0]!='.']

# choose your wanted idx
v_idx = video_idx[0]

os.makedirs('./tmp_imgs', exist_ok=True)

### Warning : check len(pkl_list) before. too much number of images can make your server's RAM explode

In [3]:
pkl_dir = os.path.join(config.skeleton_pkl_dir, v_idx)
npz_dir = os.path.join(config.skeleton_npz_dir, v_idx)

pkl_list = sorted(os.listdir(pkl_dir))
npz_list = sorted(os.listdir(npz_dir))

fig = plt.figure(figsize=(13, 7))
ax_1 = fig.add_subplot(121)
ax_2 = fig.add_subplot(122, projection='3d')

# sample one example
for ii, i in enumerate(range(len(pkl_list))):
    fp = pkl_list[i]
    bbox_fp = os.path.join(config.mocap_dir, v_idx, 'bbox', fp[:-4]+'_bbox.json')
    bbox = json.load(open(bbox_fp, 'rb'))
    img_fp = os.path.join(config.image_dir, v_idx, fp[:-4]+'.jpg')
    img = plt.imread(img_fp)

    ### get bbox size
    bbox_np = np.array(bbox['body_bbox_list'])
    bbox_size = [bb[2]*bb[3] for bb in bbox_np]

    ### only choose the biggest bbox
    max_idx = np.argmax(bbox_size)
    curr_bbox = bbox_np[max_idx]

    curr_pkl = pickle.load(open(os.path.join(pkl_dir, fp), 'rb'))
    curr_npz = np.load(os.path.join(npz_dir, fp.replace('pkl', 'npz')))

    ### [PKL] Plot 2D result
    ax_1.clear()
    ax_1.imshow(img)
    utils.plot_2D(curr_pkl['open_pose'], config.open_pose_dict, 
                  curr_pkl['left_hand'], curr_pkl['right_hand'], config.hand_dict,
                  curr_bbox, ax_1)


    ### [PKL] Plot 3D result
    ax_2.clear()

    utils.plot_3D(curr_pkl['open_pose'], curr_pkl['left_hand'], curr_pkl['right_hand'],
                  config.open_pose_dict, config.hand_dict, ax_2)

    fig.savefig('./tmp_imgs/{0:04}.jpg'.format(ii))

In [4]:
from moviepy.editor import *

tmp_img_path = ['./tmp_imgs/{0:04}.jpg'.format(x) for x in range(len(pkl_list))]

clips = [ImageClip(m).set_duration(1/10) for m in tmp_img_path]

concat_clip = concatenate_videoclips(clips, method="compose")
concat_clip.write_videofile("test_video.mp4", fps=10)


t:   0%|          | 0/301 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_video.mp4.
Moviepy - Writing video test_video.mp4



Moviepy - Done !
Moviepy - video ready test_video.mp4
